In [3]:
%load_ext jupyter_black
import os
from glob import glob
import pandas as pd
import numpy as np
import xarray as xr
import requests
import re

URLS = {
    "DATA.NOAA": "https://data.noaa.gov/onestop/",
    "NCEI.NOAA": "https://www.ncei.noaa.gov/products/",
    "NOMADS.NCEP": "https://nomads.ncep.noaa.gov/pub/",
    # realtime
    "MRMS.NCEP":"https://mrms.ncep.noaa.gov/data",
    # archive
    "MRMS.ARGON":"https://mrms.agron.iastate.edu/",
}

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [25]:
var_template = "{0}_P0_L100_GLL0"
coordinates = ["lv_ISBL0", "lat_0", "lon_0"]
variables = {var_template.format(x): x for x in ("TMP", "RH", "UGRD", "VGRD", "HGT")}  # HGT_P0_L100_GLL0

In [17]:
import pandas as pd
URL_TEMPLATE = "https://nomads.ncep.noaa.gov/pub/data/nccf/com/557ww/prod/557ww.{year:04}{month:02d}{day:02d}/GLOBAL.grib2.{year:04}{month:02d}{day:02d}{hour:02d}.{forecast_hour:04d}"


def build_url(model_run: str, forecast_hour: int = 0):
    run = pd.to_datetime(model_run)
    return URL_TEMPLATE.format(
        **{x: getattr(run, x) for x in ("year", "month", "day", "hour")}, forecast_hour=forecast_hour
    )


url = build_url("2022-05-30T00:00", forecast_hour=6)
url

'https://nomads.ncep.noaa.gov/pub/data/nccf/com/557ww/prod/557ww.20220530/GLOBAL.grib2.2022053000.0006'

In [19]:
from wxpy import download_file
f = download_file(url)
f

KeyboardInterrupt: 

In [15]:
from wxpy import download_file
url ="https://nomads.ncep.noaa.gov/pub/data/nccf/com/hrrr/v4.1/hrrr.20220530/conus/hrrr.t00z.wrfnatf01.grib2"
download_file(url)


KeyboardInterrupt: 

In [13]:
import sys

for i in range(5):
    print()

["b'",
 '!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\\n',
 'html>\\n ',
 'head>\\n  ',
 'title>Index of /pub/data',
 '/title>\\n ',
 'script>NS_CSM_td=1413533186;NS_CSM_pd=275116669;NS_CSM_u="/clm10";NS_CSM_col="Logstream";',
 '/script>',
 'script type="text/javascript">function sendTimingInfoInit(){setTimeout(sendTimingInfo,0)}function sendTimingInfo(){var wp=window.performance;if(wp){var c1,c2,t;c1=wp.timing;if(c1){var cm={};cm.ns=c1.navigationStart;if((t=c1.unloadEventStart)>0)cm.us=t;if((t=c1.unloadEventEnd)>0)cm.ue=t;if((t=c1.redirectStart)>0)cm.rs=t;if((t=c1.redirectEnd)>0)cm.re=t;cm.fs=c1.fetchStart;cm.dls=c1.domainLookupStart;cm.dle=c1.domainLookupEnd;cm.cs=c1.connectStart;cm.ce=c1.connectEnd;if((t=c1.secureConnectionStart)>0)cm.scs=t;cm.rqs=c1.requestStart;cm.rss=c1.responseStart;cm.rse=c1.responseEnd;cm.dl=c1.domLoading;cm.di=c1.domInteractive;cm.dcls=c1.domContentLoadedEventStart;cm.dcle=c1.domContentLoadedEventEnd;cm.dc=c1.domComplete;if((t=c1.loadEventStart)>0)c

In [29]:
def dataset_from_url(url) -> xr.Dataset:
    file_path = "tmp/" + re.search(r"\d*\.\d{4}$", url).group()
    r = requests.get(url)
    ds = None
    if r.status_code == 200:
        print("writing file")
        with open(file_path, "rb") as f:
            f.write(r.content)
        ds: xr.Dataset = xr.load_dataset(file_path, engine="pynio")
    else:
        print("bad status code")
    return ds


ds = dataset_from_url(url)
ds

KeyboardInterrupt: 

In [ ]:
var_template = "{0}_P0_L100_GLL0"
coordinates = ["lv_ISBL0", "lat_0", "lon_0"]
variables = {var_template.format(x): x for x in ("TMP", "RH", "UGRD", "VGRD", "HGT")}  # HGT_P0_L100_GLL0
ds[variables.keys()].rename(variables | {"lv_ISBL0": "hPa", "lat_0": "lat", "lon_0": "lon"}).to_dataframe()